In [1]:
from line_solver import *

import numpy as np
GlobalConstants.setVerbose(VerboseLevel.STD)

In [2]:

model = Network('model')
n = 5  # number of items
m = 2  # cache capacity

delay = Delay(model, 'Delay')
cacheNode = Cache(model, 'Cache', n, m, ReplacementStrategy.FIFO)
jobClass = ClosedClass(model, 'JobClass', 1, delay, 0)
hitClass = ClosedClass(model, 'HitClass', 0, delay, 0)
missClass = ClosedClass(model, 'MissClass', 0, delay, 0)

delay.setService(jobClass, Exp(1))

# Uniform item references - create Matrix from numpy array
prob_array = np.ones(n) / n
pAccess = DiscreteSampler(jlineMatrixFromArray(prob_array))
cacheNode.setRead(jobClass, pAccess)

cacheNode.setHitClass(jobClass, hitClass)
cacheNode.setMissClass(jobClass, missClass)
P = model.initRoutingMatrix()
P.set(jobClass, jobClass, delay, cacheNode, 1.0)
P.set(hitClass, jobClass, cacheNode, delay, 1.0)
P.set(missClass, jobClass, cacheNode, delay, 1.0)

model.link(P)


In [4]:
solver = np.array([], dtype=object)

# SolverCTMC
solver = np.append(solver, SolverCTMC(model, 'keep', False, 'seed', 1))
print('SOLVER: SolverCTMC')
AvgTable1 = solver[0].getAvgNodeTable()

# Reset model
model.reset()

# SolverSSA
solver = np.append(solver, SolverSSA(model, 'samples', 10000, 'verbose', True, 'method', 'serial', 'seed', 1))
print('\nSOLVER: SolverSSA')
AvgTable2 = solver[1].getAvgNodeTable()

# SolverMVA
solver = np.append(solver, SolverMVA(model, 'seed', 1))
print('\nSOLVER: SolverMVA')
AvgTable3 = solver[2].getAvgNodeTable()


SOLVER: SolverCTMC
                Node   JobClass  QLen  Util  RespT  ResidT    ArvR    Tput
0              Delay   JobClass   1.0   1.0    1.0  0.3333  0.3333  0.3333
1              Delay   HitClass   0.0   0.0    0.0  0.0000  0.3333  0.3333
2              Delay  MissClass   0.0   0.0    0.0  0.0000  0.3333  0.3333
4              Cache   HitClass   0.0   0.0    0.0  0.0000  0.0000  0.3333
5              Cache  MissClass   0.0   0.0    0.0  0.0000  0.0000  0.3333
6  CS_Cache_to_Delay   JobClass   0.0   0.0    0.0  0.0000  0.3333  1.0000
7  CS_Cache_to_Delay   HitClass   0.0   0.0    0.0  0.0000  0.3333  0.0000
8  CS_Cache_to_Delay  MissClass   0.0   0.0    0.0  0.0000  0.3333  0.0000

SOLVER: SolverSSA
SSA samples:   10000
SSA [method: serial, lang: java, env: 21.0.7] completed in 0.794000s.
                Node   JobClass  QLen  Util  RespT  ResidT  ArvR  Tput
0              Delay   JobClass   1.0   1.0    1.0     1.0   1.0   1.0
3              Cache   JobClass   0.0   0.0    0.0    